# Урок 3. Структури даних I
## Частина 3. Алгоритми для рядків
Одна з найпростіших завдань пошуку інформації - пошук точно заданого підрядка в рядку. Проте, це завдання надзвичайно важлива - вона застосовується в текстових редакторах, СУБД, пошукових машинах і т.д.

### Точна відповідність (Naive exact matching)
Дан текст t і зразок p (вважаємо, що | p | <| t |).

#### Задача: 
знайти всі входження зразка p в текст t

#### алгоритм:

i = 0,
порівняти i-й символ t з першим символом p,
збіг -> порівняти другі символи і так далі,
розбіжність -> i + = 1 і перехід до другого пункту
#### Складність в гіршому випадку: O (| t | * | p |)

In [2]:
def naive_match(p, t):
    assert len(p) <= len(t)  # assume text at least as long as pattern
    occurrences = []
    for i in range(0, len(t)-len(p)+1):  # for each alignment of p to t
        match = True  # assume we match until proven wrong
        for j in range(0, len(p)):  # for each position of p
            if t[i+j] != p[j]:
                match = False  # at least 1 char mismatches
                break
        if match:
            occurrences.append(i)
    return occurrences

In [3]:
t = 'I need a needle in a haystack' # "text" - thing we search in
p = 'needle' # "pattern" - thing we search for
naive_match(p, t)

[9]


Переконаємося, що needle дійсно знайдено

In [4]:
t[9: 9 + len(p)]

'needle'

In [5]:
naive_match('needle', 'needleneedleneedle')

[0, 6, 12]

Така складність алгоритму - недозволена розкіш для пошуку у великих текстах.

Подивимося, як заміна рядки на деякий вектор може допомогти в таких завданнях як пошук підрядка і стиснення рядки.

#### Z-функція

Нехай дана рядок s довжини n. Тоді Z-функція ( "зет-функція") від цього рядка - це масив довжини n, i-ий елемент якого дорівнює найбільшому числу символів, починаючи з позиції i, співпадаючих з першими символами рядка s.

Приклад: Z (abcdabscabcdabia) = [16,0,0,0,2,0,0,0,6,0,0,0,2,0,0,1]. Ще приклади і опис алгоритму на сайті [e-maxx](http://e-maxx.ru/algo/export_z_function).

Нижче наведено код (опціонально).

In [6]:
def z_func(s):
    
    Z = [len(s)] + [0] * len(s)
    assert len(s) > 1
    
    # Initial comparison of s[1:] with prefix
    for i in range(1, len(s)):
        if s[i] == s[i-1]:
            Z[1] += 1
        else:
            break
    
    r, l = 0, 0
    if Z[1] > 0:
        r, l = Z[1], 1
    
    for k in range(2, len(s)):
        assert Z[k] == 0
        if k > r:
            # Case 1
            for i in range(k, len(s)):
                if s[i] == s[i-k]:
                    Z[k] += 1
                else:
                    break
            r, l = k + Z[k] - 1, k
        else:
            # Case 2
            # Calculate length of beta
            nbeta = r - k + 1
            Zkp = Z[k - l]
            if nbeta > Zkp:
                # Case 2a: Zkp wins
                Z[k] = Zkp
            else:
                # Case 2b: Compare characters just past r
                nmatch = 0
                for i in range(r+1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = r - k + 1
    return Z

In [7]:
z_func('abracadabra')
#  abracadabra (11)
#     a        (1)
#       a      (1)
#         abra (4)
#            a (1)

[11, 0, 0, 1, 0, 1, 0, 4, 0, 0, 1, 0]

In [8]:
z_func('aaaaa')

[5, 4, 3, 2, 1, 0]


Застосування Z-функції:

 - Пошук підрядка в рядку
 - Кількість різних подстрок в рядку
 - Стиснення рядки
 
 #### Пошук підрядка в рядку
Нехай t - текст, p - зразок.

##### Завдання: знайти всі входження зразка p в текст t.

###### Рішення:

Утворити рядок s = p + $ + t, тобто до зразка пріпішем текст через символ-роздільник (який не зустрічається ніде в самих рядках).
Порахуємо для отриманого рядка Z-функцію
Тоді для будь-якого i в відрізку [0; length (t) -1] за відповідним значенням z [i + len (p) + 1] можна зрозуміти, чи входить зразок p в текст t, починаючи з позиції i: якщо це значення Z-функції одно length (p), то да, входить, інакше - немає.
Складність: O (len (p) + len (t))

код

In [9]:
def zMatch(p, t):
    s = p + "$" + t
    Z = z_func(s)
    occurrences = []
    for i in range(len(p) + 1, len(s)):
        if Z[i] == len(p):
            occurrences.append(i - (len(p) + 1))
    return occurrences


Ілюстрація: Текст "lambalambalam", шукаємо в ньому "lamb"

In [12]:
t, p = "lambalambalam", "lamb"
calculated_z = z_func("lamb$lambalambalam")
print(calculated_z)

[18, 0, 0, 0, 0, 4, 0, 0, 0, 0, 4, 0, 0, 0, 0, 3, 0, 0, 0]


Для першого індексу є входження:

In [14]:
print(len(p), calculated_z[0 + len(p) + 1])

4 4



Для другого - немає:

In [15]:
print(len(p), calculated_z[1 + len(p) + 1])

4 0


In [16]:
zMatch("lamb", "lambalambalam")

[0, 5]


ще приклади

In [17]:
t, p = 'haystack needle haystack', 'needle'
print(z_func(p + '$' + t))
zMatch('needle', 'haystack needle haystack')

[31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[9]

In [18]:
t, p = 'haystack needle needle', 'needle'
print(z_func(p + '$' + t))
zMatch('needle', 'haystack needle needle')

[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0]


[9, 16]

стиснення рядки
Дана рядок s довжини n.

Завдання: знайти найкоротший її "стислий" уявлення, тобто знайти такий рядок t найменшої довжини, що s можна представити у вигляді конкатенації однієї або декількох копій t.